# Parameters

In [ ]:
appid=''
disk=''
nic=''
tz=''
basedir=''
name=''

compare_appid=''
compare_basedir=''
compare_name=''

# start analysis cluster and run

In [ ]:
import findspark
findspark.init()

In [ ]:
import os
def get_py4jzip():
    spark_home=os.environ['SPARK_HOME']
    py4jzip = !ls {spark_home}/python/lib/py4j*.zip
    return py4jzip[0]

In [ ]:
import sys
print(sys.path)

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
import time
import sys
conf = (SparkConf()
    .set('spark.app.name', f'perf_analysis_{appid}')
    .set('spark.serializer','org.apache.spark.serializer.KryoSerializer')
    .set('spark.executor.instances', '4')
    .set('spark.executor.cores','4')
    .set('spark.executor.memory', '8g')
    .set('spark.driver.memory','20g')
    .set('spark.memory.offHeap.enabled','True')
    .set('spark.memory.offHeap.size','20g')
    .set('spark.executor.memoryOverhead','1g')
    .set('spark.executor.extraJavaOptions',
          '-XX:+UseParallelGC -XX:+UseParallelOldGC -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCTimeStamps')
    .set('spark.executorEnv.PYTHONPATH',f"{os.environ['SPARK_HOME']}/python:{get_py4jzip()}:{':'.join(sys.path)}")
    .set('spark.sql.inMemoryColumnarStorage.compressed','False')
    .set('spark.sql.inMemoryColumnarStorage.batchSize','100000')
    .set('spark.sql.execution.arrow.pyspark.fallback.enabled','True')
    .set('spark.sql.execution.arrow.pyspark.enabled','True')
    .set('spark.sql.execution.arrow.maxRecordsPerBatch','100000')
    .set("spark.sql.repl.eagerEval.enabled", True)
    .set("spark.sql.legacy.timeParserPolicy","LEGACY")     
    .set("spark.sql.session.timeZone", tz)
       )

sc = SparkContext(conf=conf,master='yarn')
sc.setLogLevel("ERROR")
spark = SQLContext(sc)
time.sleep(10)

In [ ]:
%%html
<style>
div.output_stderr {
background: #ffdd;
display: none;
}
</style>

# Sparklog

In [ ]:
%run ~/PAUS/sparklog.ipynb

In [ ]:
os.environ["https_proxy"] = "http://10.239.44.250:8080"
os.environ["http_proxy"] = "http://10.239.44.250:8080"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
disk_prefix=[f"'{dev}'" for dev in disk.split(',')]
nic_prefix=[f"'{dev}'" for dev in nic.split(',')]

# Content

In [ ]:
display(HTML('<a href=#App-info> 5  App info</a>'))
display(HTML('<a href=#Compare-to-previous-run> 6  Compare to previous run</a>'))
display(HTML('<a href=#Config-compare> 7  Config compare</a>'))

# App info

In [ ]:
app=Application_Run(appid, basedir=basedir)
appals=app.analysis['app']['als']

In [ ]:
appals.get_basic_state()

In [ ]:
summary=app.get_summary(disk_prefix=disk_prefix,nic_prefix=nic_prefix)
display(summary.style)

In [ ]:
app.generate_trace_view(disk_prefix=disk_prefix,nic_prefix=nic_prefix)

In [ ]:
appals.get_app_name()

In [ ]:
shuffle_df, dfx=appals.get_shuffle_stat()

In [ ]:
appals.get_app_info(disk_prefix=disk_prefix,nic_prefix=nic_prefix)

In [ ]:
appals.show_critical_path_time_breakdown().T

# Compare to previous run

In [ ]:
if compare_appid:
    compare_app=Application_Run(comapre_appid,basedir=compare_basedir)
    output=app.compare_app(rapp=compare_app,show_queryplan_diff=False,disk_prefix=disk_prefix,nic_prefix=nic_prefix)
    display(HTML(output))

# Config compare

In [ ]:
if compare_appid:
    display(comp_spark_conf(app_als, compare_app_als))

#  convert to HTML

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
# htmlname=nb_name.replace("ipynb","html")

In [ ]:
# !jupyter nbconvert --to html ./{nb_name} --no-input --output html/{htmlname} --template classic